In [1]:
import sys
from itertools import izip
from sklearn.cross_validation import KFold
import HTMLParser

train_file = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/tweebo/raw/train.txt"
#test_file = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/tweebo/raw/DevNoLabels.txt"
test_file = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/tweebo/inputs/test.nolabels.txt"

In [5]:
# Setup Twitter input files for TweeboParser
# raw data

#formated inputs for Tweebo
tweebo_input_train_file = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/tweebo/inputs/train_input.txt"
tweebo_input_test_file = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/tweebo/inputs/test_input.txt"
with open(train_file) as train:
    input_lines = train.readlines()
    tweets = []
    arr = []
    for line in input_lines:
        try:
            if line in ['\n', '\r\n']:
                sentence = ' '.join(str(x) for x in arr) 
                tweets.append(sentence.strip())
                arr = []
            else:
                (word, tag) = line.split()
                arr.append(word)
        except ValueError:
            continue
    open(tweebo_input_train_file, 'w').close()
    with open (tweebo_input_train_file, 'w') as f: 
        for tweet in tweets:
            f.write(tweet)
            f.write("\n")

with open(test_file) as test:
    input_lines = test.readlines()
    tweets = []
    arr = []
    for line in input_lines:
        try:
            if line in ['\n', '\r\n']:
                sentence = ' '.join(str(x) for x in arr) 
                tweets.append(sentence.strip())
                arr = []
            else:
                word = line.strip()
                arr.append(word)
        except ValueError:
            continue

    open(tweebo_input_test_file, 'w').close()
    with open (tweebo_input_test_file, 'w') as f: 
        for tweet in tweets:
            f.write(tweet)
            f.write("\n")            

In [2]:
# Setup Train/Test Tweebo files to extract features

train_tweebo_file = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/tweebo/inputs/train_input.txt.predict"
test_tweebo_file = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/tweebo/inputs/tweebo_test.nolabels.txt.predict"
file_to_train = test_file
file_to_train_tweebo = test_tweebo_file
result_file = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/tweebo/inputs/test.nolabels.processed.txt"
kaggle = True
with open(file_to_train) as train:
    input_lines = train.readlines()
    labeled_tweets = []
    arr = []
    for line in input_lines:
        try:
            if line in ['\n', '\r\n']:
                labeled_tweets.append(arr)
                arr = []
            else:
                if not kaggle:
                    (word, tag) = line.split()
                    arr.append((word, tag))
                else:
                    arr.append([line.strip()])
                
        except ValueError:
            continue

with open(file_to_train_tweebo) as train_tweebo:
    input_lines = train_tweebo.readlines()
    tweebo_tweets = []
    arr = []
    for line in input_lines:
        try:
            if line in ['\n', '\r\n']:
                tweebo_tweets.append(arr)
                arr = []
            else:
                (word_id, word, lemma, cpostag, postag, feats, head, dep_relation) = line.split()
                arr.append((word, word_id, postag, head, dep_relation));
        except ValueError:
            continue
            
features = []

for s1, s2 in zip(labeled_tweets, tweebo_tweets):
    tweebo_index = 0
    sentence = []
    head_diff = 0
    head_point = 0
    head_point_arr = []
    head_diff_arr = []
    for i in range(len(s1)):
        word1 = s1[i][0]
        escaped_word1 = HTMLParser.HTMLParser().unescape(s1[i][0]) 
        word2 = HTMLParser.HTMLParser().unescape(s2[tweebo_index][0]) 
        t_ind = tweebo_index
        if(word1 != word2 and escaped_word1 != word2):
            head_point = i+1
            head_point_arr.append(head_point)
            while True:
                if(word2 == word1 or word2 == escaped_word1):
                    if((int(s2[tweebo_index][3]) != -1 and t_ind != tweebo_index)):
                        t_ind = tweebo_index
                    break;
                else:
                    head_diff += 1
#                     if(int(s2[tweebo_index][3]) == -1 and int(s2[t_ind][3]) == -1):
#                         t_ind += 1
                    if((int(s2[tweebo_index][3]) != -1 and t_ind != tweebo_index)):
                        t_ind = tweebo_index
                    tweebo_index += 1
                    word2 += HTMLParser.HTMLParser().unescape(s2[tweebo_index][0])
            head_diff_arr.append(head_diff)
            
        if not kaggle:
            sentence.append([i+1, escaped_word1, s1[i][1], s2[t_ind][2], int(s2[t_ind][3]), s2[t_ind][4]])
        else:
            sentence.append([i+1, escaped_word1, s2[t_ind][2], int(s2[t_ind][3]), s2[t_ind][4]])
        tweebo_index += 1
    #head indexes reconciliation
    #print head_point_arr
    if len(head_diff_arr) > 0:
        #print sentence
        for word in sentence:
            if not kaggle:
                old_head = word[4]
            else:
                old_head = word[3]
            if(old_head == -1 or old_head == 0):
                continue

            start = old_head - (len(s2) - len(s1)) - 2
            end = old_head + (len(s2) - len(s1))

            if(start < 0):
                start = 0
            if(end > len(sentence)):
                end = len(sentence)

            old_word = s2[old_head-1][0]
            
            if(old_head < len(s2)):
                next_word = s2[old_head][0]
            else:
                next_word = ''
                
            old_word_next = old_word + next_word
            
            if(old_head > 0):
                prev_word = s2[old_head-2][0]
            else:
                prev_word = ''
                
            prev_old_word = prev_word + old_word
            
            new_word = ''

            for i in xrange(start, end):
                if(HTMLParser.HTMLParser().unescape(old_word) == sentence[i][1] \
                   or HTMLParser.HTMLParser().unescape(old_word_next) in sentence[i][1]\
                  or HTMLParser.HTMLParser().unescape(prev_old_word) in sentence[i][1]):                    
                    new_word = sentence[i]
                    break

            #print word, HTMLParser.HTMLParser().unescape(old_word), old_head
            if not kaggle:
                word[4] = new_word[0]
            else:
                word[3] = new_word[0]
    
    features.append(sentence)
    
open(result_file, 'w').close()
with open (result_file, 'w') as f: 
    for tweet in features:
        for word in tweet:
            f.write(' '.join(str(x) for x in word).strip())
            f.write("\n")
        f.write("\n")

In [18]:
# Setup Twitter POS file for OpenNLP Training

with open(pos_file) as pos:
    input_lines = pos.readlines()
    tweets = []
    arr = []
    for line in input_lines:
        try:
            if line in ['\n', '\r\n']:
                sentence = ' '.join(str(x) for x in arr) 
                tweets.append(sentence)
                arr = []
            else:
                (word, tag) = line.split()
                arr.append(word + "_" + tag)
        except ValueError:
            continue

#All Tweets
# filename = 'pos_tweets.txt'
# open(filename, 'w').close()
# with open (filename, 'w') as f: 
#     for tweet in tweets:
#         f.write(tweet)
#         f.write("\n")

#cross validation

kf = KFold(len(tweets), n_folds=5, shuffle=True)
print len(kf)

print(kf)  
index = 1
for train_index, test_index in kf:
    #print("TRAIN:", train_index, "TEST:", test_index)
    tweets_train = []
    for i in train_index:
        tweets_train.append(tweets[i])
    tweets_test = []
    for j in test_index:
        tweets_test.append(tweets[j])
    
    train_file = 'pos_tweets_train_' + str(index) + '.txt'
    open(train_file, 'w').close()
    with open (train_file, 'w') as f: 
        for tweet in tweets_train:
            f.write(tweet)
            f.write("\n")
    
    test_file = 'pos_tweets_test_' + str(index) + '.txt'
    open(test_file, 'w').close()
    with open (test_file, 'w') as f: 
        for tweet in tweets_test:
            f.write(tweet)
            f.write("\n")
    index += 1

    

5
sklearn.cross_validation.KFold(n=787, n_folds=5, shuffle=True, random_state=None)


In [11]:
#Combine Chunk and POS files
tagged_tweets = []
sentence = []

with open(pos_file) as pos, open(chunk_file) as chunk:
        i = 0
        for p, c in izip(pos, chunk):
            i += 1
            if p in ['\n', '\r\n']:
                tagged_tweets.append(sentence)
                sentence = []
            else:
                (word_p, p_tag) = p.split()
                (word_c, c_tag) = c.split()
                sentence.append((word_p, p_tag, c_tag.upper()))
#                 if(word_p == word_c):
#                     sentence.append((word_p, p_tag, c_tag))
#                 else:
#                     print (i, word_p, word_c, p_tag, c_tag)
    

# with open(pos_file) as pos:
#             input_lines = pos.readlines()
#             tags = []
#             sentence = []
#             for line in input_lines:
#                 try:
#                     if line in ['\n', '\r\n']:
#                         tags.append(sentence)
#                         sentence = []
#                     else:
#                         (word, tag) = line.split()
#                         sentence.append((word, tag, ''))
#                 except ValueError:
#                     continue  
print tagged_tweets
filename = 'pos_chunk_tweets.txt'
open(filename, 'w').close()
with open (filename, 'w') as f: 
    for tweet in tagged_tweets:
        for word in tweet:
            f.write(word[0] + " " + word[1] + " " + word[2] + "\n")
        f.write("\n")


[[('@paulwalk', 'USR', 'O'), ('It', 'PRP', 'B-NP'), ("'s", 'VBZ', 'B-VP'), ('the', 'DT', 'B-NP'), ('view', 'NN', 'I-NP'), ('from', 'IN', 'B-PP'), ('where', 'WRB', 'B-ADVP'), ('I', 'PRP', 'B-NP'), ("'m", 'VBP', 'B-VP'), ('living', 'VBG', 'I-VP'), ('for', 'IN', 'B-PP'), ('two', 'CD', 'B-NP'), ('weeks', 'NNS', 'I-NP'), ('.', '.', 'O'), ('Empire', 'NNP', 'B-NP'), ('State', 'NNP', 'I-NP'), ('Building', 'NNP', 'I-NP'), ('=', 'SYM', 'B-VP'), ('ESB', 'NNP', 'B-NP'), ('.', '.', 'O'), ('Pretty', 'RB', 'B-NP'), ('bad', 'JJ', 'I-NP'), ('storm', 'NN', 'I-NP'), ('here', 'RB', 'B-ADVP'), ('last', 'JJ', 'B-NP'), ('evening', 'NN', 'I-NP'), ('.', '.', 'O')], [('Small', 'JJ', 'B-NP'), ('Biz', 'NNP', 'I-NP'), ('Tech', 'NNP', 'I-NP'), ('Tour', 'NNP', 'I-NP'), ('2010', 'CD', 'I-NP'), ('Launches', 'VBZ', 'B-VP'), ('Five', 'CD', 'B-NP'), ('City', 'NN', 'I-NP'), ('Tour', 'NN', 'I-NP'), ('MONTCLAIR', 'NNP', 'B-NP'), ('N.J.', 'NNP', 'I-NP'), ('...:', ':', 'O'), ('The', 'DT', 'B-NP'), ('all', 'DT', 'I-NP'), ('day